In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from torch.autograd import Variable
from tqdm import tqdm, trange
from train import ClassicTrainer, ClassicTrainer_feature, ADGTrainer, NNL2Trainer
from data_fetcher import dataset, dataset_CADG, dataset_test_only

cell_names = [' '.join(line.split('-')[0].split()[1:]) for line in open('/home/songweig/sc/code/name2id39.txt').readlines()]


## NN and CDGN

In [42]:
#################### Settings ##############################
num_epochs = 100
batch_size = 256
dim1 = 1136
dim2 = 100
d_dim = 20499
dim_label = 39
dataset_name = 39
dim_domain = 40
data_path = '../processed_data'
model_path = '/home/songweig/sc/results/models/0413'
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
#################### Settings ##############################

# d_NN = dataset(data_path, batch_size, label_size=dim_label,dataset_name=dataset_name, validation=False)
# d_CDGN = dataset_CADG(data_path, batch_size, label_size=dim_label,dataset_name=dataset_name, validation=False)

model_path = '../results/models/0413'
NN_name = 'ClassicNN_6_0.2268.pyt'
CDGN_name = 'CDANN_0.02_17_0.3220_1.pyt'
# model_path = '../results/models/pancreas'
# NN_name = 'CDANN_pancreas_classicNN_d20.9980_0.7179_8.pyt'
# CDGN_name = 'CDANN_pancreas_d2_0.2_1_0.9831_0.8934_4.pyt'

log = dict()
t_CDGN = ADGTrainer(d_dim, 5, 0.05, dim1, dim2, dim_label, dim_domain, num_epochs, batch_size, use_gpu=True)
t_NN = ClassicTrainer(d_dim, dim1, dim2, dim_label, num_epochs, batch_size, use_gpu=True)

t_NN.dataset = d_NN
t_NN.D.load_state_dict(torch.load(os.path.join(model_path, NN_name)))
t_CDGN.dataset = d_CDGN
t_CDGN.D.load_state_dict(torch.load(os.path.join(model_path, CDGN_name)))

gene_names = np.load('/home/songweig/sc/processed_data/gene_symbols_0413.npy')
cell_names = [' '.join(line.split('-')[0].split()[1:]) for line in open('/home/songweig/sc/code/name2id39.txt').readlines()]
# gene_names = [line.split(',')[1].strip('"\n') for line in open('/home/songweig/sc/processed_data/genename_pancreas.csv')][1:]
# cell_names = [line.split('-')[0] for line in open('/home/songweig/sc/code/name2id13.txt').readlines()]

DANN_Siamese(
  (feature_extractor): Sequential(
    (0): Linear(in_features=20499, out_features=1136, bias=True)
    (1): Tanh()
    (2): Linear(in_features=1136, out_features=100, bias=True)
    (3): Tanh()
  )
  (domain_classifier): Sequential(
    (0): Linear(in_features=100, out_features=40, bias=True)
  )
  (label_classifier): Sequential(
    (0): Linear(in_features=100, out_features=39, bias=True)
  )
)
ClassicNN(
  (h1): Sequential(
    (0): Linear(in_features=20499, out_features=1136, bias=True)
    (1): Tanh()
  )
  (h2): Sequential(
    (0): Linear(in_features=1136, out_features=100, bias=True)
    (1): Tanh()
  )
  (o): Sequential(
    (0): Linear(in_features=100, out_features=39, bias=True)
  )
)


In [49]:
# extract the mean expression values
n_genes = 100
out_path = '/home/songweig/sc/results/GO/0413'
for cate_id in range(39):
    counts = 0.
#     mean_value_ori = t_CDGN.dataset._train_X.mean(0)
    mean_value_ori = np.zeros(20499)
    # for x, _, y, _, _, _, _ in t_CDGN.dataset.train_data():
    #     ids = np.where(y==cate_id)[0]
    #     counts += ids.shape[0]
    #     mean_value += x[ids].sum(0)

    # mean_value_ori = mean_value/counts
    # mean_value_ori = np.zeros(3000)

    # analysis NN: backpropogate the gradient of beta and quiescent_stellate 
    mean_value = np.copy(mean_value_ori)
    resulted_values_NN = []
    for i in range(100):
        t_NN.D.zero_grad()
        mean_value_variable = Variable(torch.Tensor(mean_value).view(1, -1).cuda(), requires_grad=True)
        act_value = t_NN.D.forward(mean_value_variable)
    #     act_value = torch.softmax(act_value, 1)
        diff_out = act_value[:,cate_id:cate_id+1]
        diff_out.backward()
        mean_value_variable.data.add_(mean_value_variable.grad.data)
        mean_value = mean_value_variable.data.cpu().numpy()
        resulted_values_NN.append(mean_value)

    # analysis CDGNN: backpropogate the gradient of beta and quiescent_stellate 
    mean_value = np.copy(mean_value_ori)
    resulted_values_CDGN = []
    for i in range(100):
        t_CDGN.D.zero_grad()
        mean_value_variable = Variable(torch.Tensor(mean_value).view(1, -1).cuda(), requires_grad=True)
        act_value, _, _ = t_CDGN.D.forward(mean_value_variable, mean_value_variable)
    #     act_value = torch.softmax(act_value, 1)
        diff_out = act_value[:,cate_id:cate_id+1]
        diff_out.backward()
        mean_value_variable.data.add_(mean_value_variable.grad.data)
        mean_value = mean_value_variable.data.cpu().numpy()
        resulted_values_CDGN.append(mean_value)

    # save DE gene names 
    cell_type = cell_names[cate_id]
    with open(os.path.join(out_path, 'NN_%s_%d.txt'%(cell_type, n_genes)), 'w') as fw:
        diff_NN = (resulted_values_NN[-1]-mean_value_ori)[0]
        diff_NN_ids = np.abs(diff_NN).argsort()[-n_genes:][::-1]
        for index in diff_NN_ids:
    #         fw.write('%s\t%.4f\n'%(gene_names[index], diff_NN[index]))
            fw.write('%s\n'%(gene_names[index]))

    with open(os.path.join(out_path, 'CDGN_%s_%d.txt'%(cell_type, n_genes)), 'w') as fw:
        diff_CDGN = (resulted_values_CDGN[-1]-mean_value_ori)[0]
        diff_CDGN_ids = np.abs(diff_CDGN).argsort()[-n_genes:][::-1]
        for index in diff_CDGN_ids:
    #         fw.write('%s\t%.4f\n'%(gene_names[index], diff_CDGN[index]))
            fw.write('%s\n'%(gene_names[index]))

    with open(os.path.join(out_path, 'Diff_%s_%d.txt'%(cell_type, n_genes)), 'w') as fw:
        diff_ids = np.setdiff1d(diff_CDGN_ids, diff_NN_ids)
        for index in diff_ids:
            fw.write('%s\n'%(gene_names[index]))

In [48]:
t_NN.dataset._train_y.shape

(37697,)

In [18]:
t_CDGN.D.zero_grad()
t_NN.D.zero_grad()
mean_value_variable = Variable(torch.Tensor(mean_value).view(1, -1).cuda(), requires_grad=True)
act_value, _, _ = t_CDGN.D.forward(mean_value_variable, mean_value_variable)
act_value.norm(1).backward()
mean_value_variable.grad.data.sum()

tensor(-4.5405, device='cuda:0')

In [5]:
# find the common important genes
mean_value_ori = np.zeros(20499)

for cate_id in range(39):
    # analysis NN: backpropogate the gradient of beta and quiescent_stellate 
    mean_value = np.copy(mean_value_ori)
    resulted_values_NN = []
    for i in range(100):
        t_NN.D.zero_grad()
        mean_value_variable = Variable(torch.Tensor(mean_value).view(1, -1).cuda(), requires_grad=True)
        act_value = t_NN.D.forward(mean_value_variable)
    #     act_value = torch.softmax(act_value, 1)
        diff_out = act_value[:,cate_id:cate_id+1]
        diff_out.backward()
        mean_value_variable.data.add_(mean_value_variable.grad.data)
        mean_value = mean_value_variable.data.cpu().numpy()
        resulted_values_NN.append(mean_value)

    # analysis CDGNN: backpropogate the gradient of beta and quiescent_stellate 
    mean_value = np.copy(mean_value_ori)
    resulted_values_CDGN = []
    for i in range(100):
        t_CDGN.D.zero_grad()
        mean_value_variable = Variable(torch.Tensor(mean_value).view(1, -1).cuda(), requires_grad=True)
        act_value, _, _ = t_CDGN.D.forward(mean_value_variable, mean_value_variable)
    #     act_value = torch.softmax(act_value, 1)
        diff_out = act_value[:,cate_id:cate_id+1]
        diff_out.backward()
        mean_value_variable.data.add_(mean_value_variable.grad.data)
        mean_value = mean_value_variable.data.cpu().numpy()
        resulted_values_CDGN.append(mean_value)
    
    diff_NN = (resulted_values_NN[-1]-mean_value_ori)[0]
    diff_NN_ids = np.abs(diff_NN).argsort()[-200:][::-1]
    diff_CDGN = (resulted_values_CDGN[-1]-mean_value_ori)[0]
    diff_CDGN_ids = np.abs(diff_CDGN).argsort()[-200:][::-1]
    if cate_id == 0:
        NN_genes = diff_NN_ids
        CDGN_genes = diff_CDGN_ids
    else:
        NN_genes = np.intersect1d(NN_genes, diff_NN_ids)
        CDGN_genes = np.intersect1d(CDGN_genes, diff_CDGN_ids)
    print(cate_id, NN_genes.shape, CDGN_genes.shape)
    

0 (200,) (200,)
1 (43,) (61,)
2 (17,) (20,)
3 (5,) (9,)
4 (2,) (5,)
5 (1,) (3,)
6 (0,) (2,)
7 (0,) (1,)


KeyboardInterrupt: 

In [ ]:

with open(os.path.join(out_path, '0Diff_%s_%d.txt'%(cell_type, n_genes)), 'w') as fw:
    diff_ids = np.setdiff1d(diff_CDGN_ids, diff_NN_ids)
    for index in diff_ids:
        fw.write('%s\n'%(gene_names[index]))